# Reducing Karatsuba multiplication to multiplication of linear polynomials

There are two key ideas to Karatsuba multiplication:

1) Naive multiplication of two two-word numbers requires 4 word multiplications, but we can get that down to 3.
2) We can propagate that savings down recursively to reduce the asymptotic running time.

\#1 is the more interesting one, especially once we jump to Toom-Cook. So let's set aside \#2 for now, and pretend that we only have to implement 2x2 multiplication.

Furthermore, note that multiplication of numbers and multiplication of polynomials are very similar -- the only difference is that for numbers we have to propagate carries, and we don't have any such requirement for polynomials, so the latter is simpler. For example, take the following simple multiplication of two 2-decimal digit numbers:

In [1]:
12 * 34

408

We can expand both operands and the result in base-10:

In [2]:
(2 + 1*10) * (4 + 3*10), 8 + 0*10 + 4*100

(408, 408)

Now let's look at the multiplication of the analogous polynomials:

In [3]:
from sympy import poly
from sympy.abc import x

p = poly(2 + 1*x) * poly(4 + 3*x)
p

Poly(3*x**2 + 10*x + 8, x, domain='ZZ')

We see that the polynomial looks like 408, except that the "tens digit", i.e. the coefficient of x, remains as 10 instead of being 0 and carrying the 1 to the "hundreds digit". In fact, if we evaluate the polynomial at the base (10), we get our numerical result back:

In [4]:
 p(10)

408

Therefore, we can get the main idea of Karatsuba multiplication by just considering the multiplication of two polynomials of degree at most 1. Here's the schoolbook version that requires 4 coefficient multiplications:

In [5]:
# a and b are lists of polynomial coefficients, with a[i] being the coefficient of x**i.
def schoolbook_mult_2x2_poly(a, b):
    c = [0] * 3
    c[0] = a[0] * b[0]
    c[1] = a[1] * b[0] + a[0] * b[1]
    c[2] = a[1] * b[1]
    return c

Plugging our example above, we get the same result (note that the list of coefficients goes from smallest to largest degree, which is the opposite from how we normally write polynomials or numbers).

In [6]:
schoolbook_mult_2x2_poly([2, 1], [4, 3])

[8, 10, 3]

Now, if we stare at `schoolbook_mult_2x2_poly` for a while, and if we're told that it's possible to evaluate the elements of `c` using only 3 multiplications, `c[1]` would be a natural place to start. Then we could notice that the terms of `c[1]` appear as the cross-terms of `(a[0] + a[1])*(b[0] + b[1])`, i.e. the middle terms of the expansion:
```
a[0]*b[0] + a[1]*b[0] + a[0]*b[1] + a[1]*b[1]
```
and also, that the first and last terms are nothing but `c[0]` and `c[2]`. Then that gives us the first form of Karatsuba multiplication:

In [7]:
def karatsuba_mult_poly(a, b):
    c = [0] * 3
    c[0] = a[0] * b[0]
    c[2] = a[1] * b[1]
    t = (a[0] + a[1]) * (b[0] + b[1])
    c[1] = t - c[0] - c[2]
    return c

karatsuba_mult_poly([2, 1], [4, 3])

[8, 10, 3]

[Modern Computer Arithmetic](https://maths-people.anu.edu.au/~brent/pd/mca-cup-0.5.9.pdf) (\[MCA\]) gives a slightly more complicated form in order to avoid carries -- that doesn't matter for our case since we're doing polynomial multiplication, but here is their version:

In [8]:
def karatsuba_mult_poly_alt(a, b):
    c = [0] * 3
    c[0] = a[0] * b[0]

    # Note that [MCA] calls this C_1
    c[2] = a[1] * b[1]
    
    s_a = +1 if a[0] > a[1] else -1
    s_b = +1 if b[0] > b[1] else -1
    t = abs(a[0] - a[1]) * abs(b[0] - b[1])
    
    # Note that [MCA] calls this C_2.
    c[1] = c[0] + c[2] - s_a * s_b * t
    return c

karatsuba_mult_poly_alt([2, 1], [4, 3])

[8, 10, 3]

Now we can move on to the more complicated case of 3x3 multiplication, which leads us to Toom-Cook multiplication.

# Reducing Toom-Cook 3x3 multiplication to multiplication of quadratic polynomials

Similar to what we did above, for Toom-Cook 3x3, we can look at multiplication of 3-word numbers, and then to multiplication of two polynomials of degree at most 2. Here's our numerical example:

In [9]:
123 * 456

56088

And here's our polynomial example:

In [10]:
def schoolbook_mult3x3_poly(a, b):
    c = [0] * 5
    c[0] = a[0] * b[0]
    c[1] = a[1] * b[0] + a[0] * b[1]
    c[2] = a[2] * b[0] + a[1] * b[1] + a[0] * b[2]
    c[3] = a[2] * b[1] + a[1] * b[2]
    c[4] = a[2] * b[2]
    return c

schoolbook_mult3x3_poly([3, 2, 1], [6, 5, 4])

[18, 27, 28, 13, 4]

Now the main tool that Toom-Cook uses is the Lagrange interpolation theorem, which in one form states:

Given rational points $(x_0, y_0), \dotsc, (x_n, y_n)$ such that all $x_i$ are distinct, there exists a unique polynomial $p$ of degree at most $n$ such that $p(x_i) = y_i$ for all $i$.

For now, we'll treat this theorem like a black box, and assume that, say, SymPy's solver "knows" how to find this unique polynomial. Below, we'll only need this theorem for $n = 4$, so we'll just implement that:

In [11]:
from sympy import solve, symbols

def poly_from_coeffs(coeffs):
    return poly(sum(c * (x**i) for i, c in enumerate(coeffs)))

def lagrange_interpolate_deg4_v0(points):
    (x0, y0), (x1, y1), (x2, y2), (x3, y3), (x4, y4) = points

    # We're solving for the polynomial
    #
    #   p(x) = a0 + a1 * x + a2 * x**2 + a3 * x**3 + a4 * x**4,
    #
    # i.e. for the coefficients a0, a1, a2, a3, a4.
    a0, a1, a2, a3, a4 = symbols('a0 a1 a2 a3 a4')
    s = solve([
        a0 * x0**0 + a1 * x0**1 + a2 * x0**2 + a3 * x0**3 + a4 * x0**4 - y0,
        a0 * x1**0 + a1 * x1**1 + a2 * x1**2 + a3 * x1**3 + a4 * x1**4 - y1,
        a0 * x2**0 + a1 * x2**1 + a2 * x2**2 + a3 * x2**3 + a4 * x2**4 - y2,
        a0 * x3**0 + a1 * x3**1 + a2 * x3**2 + a3 * x3**3 + a4 * x3**4 - y3,
        a0 * x4**0 + a1 * x4**1 + a2 * x4**2 + a3 * x4**3 + a4 * x4**4 - y4,
    ])
    coeffs = [s[a0], s[a1], s[a2], s[a3], s[a4]]
    return poly_from_coeffs(coeffs)

# Should result in p0(x) = x**4 + 5.
p0 = lagrange_interpolate_deg4_v0([(0, 0 + 5), (1, 1 + 5), (2, 16 + 5), (3, 81 + 5), (4, 256 + 5)])

# Should result in p1(x) = x.
p1 = lagrange_interpolate_deg4_v0([(1, 1), (2, 2), (3, 3), (4, 4), (5, 5)])

p0, p1

(Poly(x**4 + 5, x, domain='ZZ'), Poly(x, x, domain='ZZ'))

How does this help us? Well, if we have two (up to) quadratic polynomials $a$ and $b$ and we want to find their product $c$ which would be at most a quartic (degree-4) polynomial, we can:
1) evaluate $a$ and $b$ at 5 distinct values $x_0, \dotsc, x_4$ to get $a(x_0), \dotsc, a(x_4)$ and $b(x_0), \dotsc, b(x_4)$,
2) multiply each $a(x_i)$ and $b(x_i)$ to get $y_i = c(x_i)$,
3) find the Lagrange interpolation polynomial for $(x_0, y_0), \dotsc, (x_4, y_4)$, which by uniqueness would have to be $c$.

This is what we implement below, picking the arbitrary values $-2, -1, 0, 1, 2$ (we'll examine how the choice of values influences performance below):

In [12]:
def toom_cook_mult3x3_poly_v0(a, b):
    xs = [-2, -1, 0, 1, 2]
    axs = [poly_from_coeffs(a).eval(x) for x in xs]
    bxs = [poly_from_coeffs(b).eval(x) for x in xs]
    cxs = [ax * bx for ax, bx in zip(axs, bxs)]
    
    # all_coeffs returns the coefficients from largest degree down.
    return list(reversed(lagrange_interpolate_deg4_v0(zip(xs, cxs)).all_coeffs()))

toom_cook_mult3x3_poly_v0([3, 2, 1], [6, 5, 4])

[18, 27, 28, 13, 4]

Now of course, we can't run a solver for every multiplication, so we need to implement Lagrange interpolation ourselves. One thing to notice is that the system of equations in `lagrange_interpolate_deg4_v0` is linear in `a0, a1, a2, a3, a4`, so we can cast it as a matrix equation:

In [13]:
from sympy.matrices import Matrix

def lagrange_interpolate_deg4_v1(points):
    (x0, y0), (x1, y1), (x2, y2), (x3, y3), (x4, y4) = points
    
    A = Matrix([
        [x0**0, x0**1, x0**2, x0**3, x0**4],
        [x1**0, x1**1, x1**2, x1**3, x1**4],
        [x2**0, x2**1, x2**2, x2**3, x2**4],
        [x3**0, x3**1, x3**2, x3**3, x3**4],
        [x4**0, x4**1, x4**2, x4**3, x4**4],
    ])
    b = Matrix([y0, y1, y2, y3, y4])
    # Solve the matrix equation A * [c0, ..., c4]^T = [y0, .., y4]^T.
    coeffs = A.solve(b)
    return poly_from_coeffs(coeffs)

# Should result in p0(x) = x**4 + 5.
p0 = lagrange_interpolate_deg4_v0([(0, 0 + 5), (1, 1 + 5), (2, 16 + 5), (3, 81 + 5), (4, 256 + 5)])

# Should result in p1(x) = x.
p1 = lagrange_interpolate_deg4_v0([(1, 1), (2, 2), (3, 3), (4, 4), (5, 5)])

p0, p1

(Poly(x**4 + 5, x, domain='ZZ'), Poly(x, x, domain='ZZ'))

Another thing to notice is that it is easy to compute the leading term of the product of two polynomials -- it's just the product of the leading terms of the two polynomials! Given that, we can "substitute" the leading term for one of the points that Lagrange interpolation requires by replacing one of the rows:

In [14]:
def lagrange_interpolate_coeffs_deg4_with_leading_term(points, c4):
    (x0, y0), (x1, y1), (x2, y2), (x3, y3) = points
    A = Matrix([
        [x0**0, x0**1, x0**2, x0**3, x0**4],
        [x1**0, x1**1, x1**2, x1**3, x1**4],
        [x2**0, x2**1, x2**2, x2**3, x2**4],
        [x3**0, x3**1, x3**2, x3**3, x3**4],
        # Equivalent to saying c4 (what we're solving for) = c4 (what we passed in).
        [0, 0, 0, 0, 1],
    ])
    b = Matrix([y0, y1, y2, y3, c4])
    return poly_from_coeffs(A.solve(b))

# Should result in p0(x) = x**4 + 5.
p0 = lagrange_interpolate_coeffs_deg4_with_leading_term([(0, 0 + 5), (1, 1 + 5), (2, 16 + 5), (3, 81 + 5)], 1)

# Should result in a complicated cubic polynomial, since we're "forcing" the leading term to be 0..
p1 = lagrange_interpolate_coeffs_deg4_with_leading_term([(0, 0 + 5), (1, 1 + 5), (2, 16 + 5), (3, 81 + 5)], 0)

# Should result in p2(x) = x.
p2 = lagrange_interpolate_coeffs_deg4_with_leading_term([(1, 1), (2, 2), (3, 3), (4, 4)], 0)

# Results in a complicated polynomial, since we're "forcing" the leading term to be x**4.
p3 = lagrange_interpolate_coeffs_deg4_with_leading_term([(1, 1), (2, 2), (3, 3), (4, 4)], 1)

p0, p1, p2, p3

(Poly(x**4 + 5, x, domain='ZZ'),
 Poly(6*x**3 - 11*x**2 + 6*x + 5, x, domain='ZZ'),
 Poly(x, x, domain='ZZ'),
 Poly(x**4 - 10*x**3 + 35*x**2 - 49*x + 24, x, domain='ZZ'))

Then we can re-implement Toom-Cook with this slightly simpler solver -- the main procedure still remains the same.

In [15]:
def toom_cook_mult3x3_poly_v1(a, b):
    xs = [-1, 0, 1, 2]
    axs = [poly_from_coeffs(a).eval(x) for x in xs]
    bxs = [poly_from_coeffs(b).eval(x) for x in xs]
    cxs = [ax * bx for ax, bx in zip(axs, bxs)]
    c4 = a[2] * b[2]
    
    # all_coeffs returns the coefficients from largest degree down.
    return list(reversed(lagrange_interpolate_coeffs_deg4_with_leading_term(zip(xs, cxs), c4).all_coeffs()))

toom_cook_mult3x3_poly_v1([3, 2, 1], [6, 5, 4])

[18, 27, 28, 13, 4]

The `xs` are simple constants, so we may as well manually compute the coefficients of `a` and `b`, rather than converting them to polynomials and evaluating.

In [16]:
def toom_cook_mult3x3_poly_v2(a, b):
    # a(-1) and b(-1)
    a_n1 = a[0] - a[1] + a[2]
    b_n1 = b[0] - b[1] + b[2]
    # a(0) and b(0)
    a_0 = a[0]
    b_0 = b[0]
    # a(1) and b(1)
    a_1 = a[0] + a[1] + a[2]
    b_1 = b[0] + b[1] + b[2]
    # a(2) and b(2)
    a_2 = a[0] + 2*a[1] + 4*a[2]
    b_2 = b[0] + 2*b[1] + 4*b[2]
    
    # c(-1), c(0), c(1), and c(2)
    c_n1 = a_n1 * b_n1
    c_0 = a_0 * b_0
    c_1 = a_1 * b_1
    c_2 = a_2 * b_2
    
    # convention: c_i means c(i) (c evaluated at i), whereas ci means the ith coefficient of c.
    c4 = a[2] * b[2]
    
    # all_coeffs returns the coefficients from largest degree down.
    return list(reversed(lagrange_interpolate_coeffs_deg4_with_leading_term([(-1, c_n1), (0, c_0), (1, c_1), (2, c_2)], c4).all_coeffs()))

toom_cook_mult3x3_poly_v2([3, 2, 1], [6, 5, 4])

[18, 27, 28, 13, 4]

Note that the choice of interpolation values influences the ease of computation of the coefficients of `a` and `b` -- we almost certainly want to use $0$, as that immediately gets us the trailing term `c0 == c_0`, similar to how we can compute the leading term `c4` immediately. But `+1` and `-1` avoids multiplication entirely, whereas `2` only involves multiplication by `2` and `4`, which can be done using bitshifts. We'll also see later how it influences later calculations.

Since the `xs` are constants, we may as well bake those constants into the Lagrange interpolation. We may as well skip the conversions to and from a polynomial while we're at it:

In [17]:
def lagrange_interpolate_coeffs_at_n1_0_1_2_with_leading_term_v0(y_n1, y0, y1, y2, c4):
    A = Matrix([
        # -1**0, (-1)**1, (-1)**2, (-1)**3, (-1)**4
        [1, -1, 1, -1, 1],
        # 0**0 = 1, 0**1, 0**2, 0**3, 0**4
        [1, 0, 0, 0, 0],
        # 1**0, 1**1, 1**2, 1**3, 1**4
        [1, 1, 1, 1, 1],
        # 2**0, 2**1, 2**2, 2**3, 2**4
        [1, 2, 4, 8, 16],
        # Equivalent to saying c4 (what we're solving for) = c4 (what we passed in).
        [0, 0, 0, 0, 1],
    ])
    b = Matrix([y_n1, y0, y1, y2, c4])
    return A.solve(b)

def toom_cook_mult3x3_poly_v3(a, b):
    # a(-1) and b(-1)
    a_n1 = a[0] - a[1] + a[2]
    b_n1 = b[0] - b[1] + b[2]
    # a(0) and b(0)
    a_0 = a[0]
    b_0 = b[0]
    # a(1) and b(1)
    a_1 = a[0] + a[1] + a[2]
    b_1 = b[0] + b[1] + b[2]
    # a(2) and b(2)
    a_2 = a[0] + 2*a[1] + 4*a[2]
    b_2 = b[0] + 2*b[1] + 4*b[2]
    
    # c(-1), c(0), c(1), and c(2)
    c_n1 = a_n1 * b_n1
    c_0 = a_0 * b_0
    c_1 = a_1 * b_1
    c_2 = a_2 * b_2
    
    # convention: c_i means c(i) (c evaluated at i), whereas ci means the ith coefficient of c.
    c4 = a[2] * b[2]
    
    return lagrange_interpolate_coeffs_at_n1_0_1_2_with_leading_term_v0(c_n1, c_0, c_1, c_2, c4)

toom_cook_mult3x3_poly_v3([3, 2, 1], [6, 5, 4])

Matrix([
[18],
[27],
[28],
[13],
[ 4]])

Then, notice that we have a constant matrix `A` in `lagrange_interpolate_coeffs_at_n1_0_1_2_with_leading_term_v0` (incidentally, this is a type of what is called a Vandermonde matrix). Then we can just pre-compute its inverse:

In [18]:
A = Matrix([
    [1, -1, 1, -1, 1],
    [1, 0, 0, 0, 0],
    [1, 1, 1, 1, 1],
    [1, 2, 4, 8, 16],
    [0, 0, 0, 0, 1],
])
A.inv()

Matrix([
[   0,    1,    0,    0,  0],
[-1/3, -1/2,    1, -1/6,  2],
[ 1/2,   -1,  1/2,    0, -1],
[-1/6,  1/2, -1/2,  1/6, -2],
[   0,    0,    0,    0,  1]])

Since `lagrange_interpolate_coeffs_at_n1_0_1_2_with_leading_term_v1` is now just a multiplication by a constant matrix, we may as well just inline that into our Toom-Cook algorithm:

In [19]:
from sympy import Rational as R

def toom_cook_mult3x3_poly_v4(a, b):
    # a(-1) and b(-1)
    a_n1 = a[0] - a[1] + a[2]
    b_n1 = b[0] - b[1] + b[2]
    # a(0) and b(0)
    a_0 = a[0]
    b_0 = b[0]
    # a(1) and b(1)
    a_1 = a[0] + a[1] + a[2]
    b_1 = b[0] + b[1] + b[2]
    # a(2) and b(2)
    a_2 = a[0] + 2*a[1] + 4*a[2]
    b_2 = b[0] + 2*b[1] + 4*b[2]
    
    # c(-1), c(0), c(1), and c(2)
    c_n1 = a_n1 * b_n1
    c_0 = a_0 * b_0
    c_1 = a_1 * b_1
    c_2 = a_2 * b_2
    
    # convention: c_i means c(i) (c evaluated at i), whereas ci means the ith coefficient of c.
    c4 = a[2] * b[2]
    
    A_inv = Matrix([
        [0, 1, 0, 0, 0],
        [R(-1, 3), R(-1, 2), 1, R(-1, 6), 2],
        [R(1, 2), -1, R(1, 2), 0, -1],
        [R(-1, 6), R(1, 2), R(-1, 2), R(1, 6), -2],
        [0, 0, 0, 0, 1],
    ])
    b = Matrix([c_n1, c_0, c_1, c_2, c4])
    return A_inv * b

toom_cook_mult3x3_poly_v4([3, 2, 1], [6, 5, 4])

Matrix([
[18],
[27],
[28],
[13],
[ 4]])

Then we may as well do inline the matrix multiplication ourselves; note that this now has no external dependencies, and is starting to look like the algorithm given in \[MCA\]:

In [20]:
def toom_cook_mult3x3_poly_v5(a, b):
    # a(-1) and b(-1)
    a_n1 = a[0] - a[1] + a[2]
    b_n1 = b[0] - b[1] + b[2]
    # a(0) and b(0)
    a_0 = a[0]
    b_0 = b[0]
    # a(1) and b(1)
    a_1 = a[0] + a[1] + a[2]
    b_1 = b[0] + b[1] + b[2]
    # a(2) and b(2)
    a_2 = a[0] + 2*a[1] + 4*a[2]
    b_2 = b[0] + 2*b[1] + 4*b[2]
    
    # c(-1), c(0), c(1), and c(2)
    c_n1 = a_n1 * b_n1
    c_0 = a_0 * b_0
    c_1 = a_1 * b_1
    c_2 = a_2 * b_2
    
    # convention: c_i means c(i) (c evaluated at i), whereas ci means the ith coefficient of c.
    c4 = a[2] * b[2]
    
    c1 = R(-c_n1, 3) + R(-c_0, 2) + c_1 + R(-c_2, 6) + 2*c4
    c2 = R(c_n1, 2) - c_0 + R(c_1, 2) - c4
    c3 = R(-c_n1, 6) + R(c_0, 2) + R(-c_1, 2) + R(c_2, 6) - 2*c4
    return [c_0, c1, c2, c3, c4]

toom_cook_mult3x3_poly_v5([3, 2, 1], [6, 5, 4])

[18, 27, 28, 13, 4]

Now we see how the choice of interpolation values influences computation of `c1`, `c2`, and `c3` -- the interpolation values determine the entries of the matrix $A$, which in turn determines the entries of the matrix $A^{-1}$, which then determine `c1`, `c2`, and `c3`.

The divisions by 2 aren't a big deal, since they're just bitshifts, but one might wonder if the division by 3 (and 6, which would be a bitshift plus a division by 3) is avoidable. It is in fact unavoidable, due to properties of Vandermonde matrices and their determinants -- this is Exercise 1.14 in \[MCA\] (we also prove it below).

But if the division by 3 is unavoidable, we may as well try to minimize them, ideally only doing them once, by trying to combine common subexpressions. Here's my attempt:

In [21]:
def toom_cook_mult3x3_poly_v6(a, b):
    # a(-1) and b(-1)
    a_n1 = a[0] - a[1] + a[2]
    b_n1 = b[0] - b[1] + b[2]
    # a(0) and b(0)
    a_0 = a[0]
    b_0 = b[0]
    # a(1) and b(1)
    a_1 = a[0] + a[1] + a[2]
    b_1 = b[0] + b[1] + b[2]
    # a(2) and b(2)
    a_2 = a[0] + 2*a[1] + 4*a[2]
    b_2 = b[0] + 2*b[1] + 4*b[2]
    
    # c(-1), c(0), c(1), and c(2)
    c_n1 = a_n1 * b_n1
    c_0 = a_0 * b_0
    c_1 = a_1 * b_1
    c_2 = a_2 * b_2
    
    # convention: c_i means c(i) (c evaluated at i), whereas ci means the ith coefficient of c.
    c4 = a[2] * b[2]
    
    c_0_half = R(c_0, 2)
    c_1_half = R(c_1, 2)
    t0 = R(c_2, 6) - 2*c4
    
    c1 = -c_0_half - R(c_n1, 3) + c_1 - t0
    c2 = R(c_n1, 2) - c_0 + c_1_half - c4
    c3 = c_0_half - R(c_n1, 6) - c_1_half + t0
    return [c_0, c1, c2, c3, c4]

toom_cook_mult3x3_poly_v6([3, 2, 1], [6, 5, 4])

[18, 27, 28, 13, 4]

I then cheated and looked at what \[MCA\] did. Note that this is where it becomes more of a black art than a systematic process -- we want to choose the `x_i`s and operations carefully to minimize the number of divisions.

In [22]:
def toom_cook_mult3x3_poly_v7(a, b):
    # a(-1) and b(-1)
    a_n1 = a[0] - a[1] + a[2]
    b_n1 = b[0] - b[1] + b[2]
    # a(0) and b(0)
    a_0 = a[0]
    b_0 = b[0]
    # a(1) and b(1)
    a_1 = a[0] + a[1] + a[2]
    b_1 = b[0] + b[1] + b[2]
    # a(2) and b(2)
    a_2 = a[0] + 2*a[1] + 4*a[2]
    b_2 = b[0] + 2*b[1] + 4*b[2]
    
    # c(-1), c(0), c(1), and c(2)
    c_n1 = a_n1 * b_n1
    c_0 = a_0 * b_0
    c_1 = a_1 * b_1
    c_2 = a_2 * b_2
    
    # convention: c_i means c(i) (c evaluated at i), whereas ci means the ith coefficient of c.
    c4 = a[2] * b[2]
    
    t1 = R(3*c_0 + 2 * c_n1 + c_2, 6) - 2*c4
    t2 = R(c_1 + c_n1, 2)
    
    c1 = c_1 - t1
    c2 = t2 - c_0 - c4
    c3 = t1 - t2
    return [c_0, c1, c2, c3, c4]

toom_cook_mult3x3_poly_v7([3, 2, 1], [6, 5, 4])

[18, 27, 28, 13, 4]

Here's the algorithm from \[MCA\], for comparison (which also eliminates a few common subexpressions in the initial computations).

In [23]:
def toom_cook_mult3x3_poly_mca(a, b):
    a02 = a[0] + a[2]
    b02 = b[0] + b[2]
    a_n1 = a02 - a[1]
    b_n1 = b02 - b[1]
    a_0 = a[0]
    b_0 = b[0]
    a_1 = a02 + a[1]
    b_1 = b02 + b[1]
    a_2 = a[0] + 2*a[1] + 4*a[2]
    b_2 = b[0] + 2*b[1] + 4*b[2]
    
    c_n1 = a_n1 * b_n1
    c_0 = a_0 * b_0
    c_1 = a_1 * b_1
    c_2 = a_2 * b_2

    # Note that [MCA] calls this v_∞, but it really is just the leading coefficient.
    c4 = a[2] * b[2]
    
    t1 = R(3*c_0 + 2*c_n1 + c_2, 6) - 2*c4
    t2 = R(c_1 + c_n1, 2)
    
    c1 = c_1 - t1
    c2 = t2 - c_0 - c4
    c3 = t1 - t2
    return [c_0, c1, c2, c3, c4]

toom_cook_mult3x3_poly_mca([3, 2, 1], [6, 5, 4])

[18, 27, 28, 13, 4]

Note that instead of 9 coefficient multiplications, we now have only 5, but we also have multiplications by constants, as well as divisions. Some of the multiplications and divisions would translate to shifts, since they're by powers of two, but there's at least one multiplication by 3, and one division by 6 (which can be done as a shift right followed by a division by 3).

# Karatsuba as a special case of Toom-Cook

From the analysis above, it should be clear that we can do something similar for $n \times n$-word multiplications, and even $m \times n$-word multiplications. But first, let's apply the same procedure to $2 \times 2$ multiplication.

Recall that we were multiplying (up to) linear polynomials, which would result in at most a quadratic (degree-2) polynomial. Therefore, that would require 2 points to interpolate, plus the leading coefficient.

A natural choice would be $x_0 = 0, x_1 = 1$. Then the (almost) Vandermonde matrix would be

$$
\left(\begin{matrix}
1 & 0 & 0 \\
1 & 1 & 1 \\
0 & 0 & 1
\end{matrix}\right)
$$

which has inverse

$$
\left(\begin{matrix}
1 & 0 & 0 \\
-1 & 1 & -1 \\
0 & 0 & 1
\end{matrix}\right)
$$

If you use this matrix, you end up with the original `karatsuba_mult_poly` algorithm. Then (I'd have to check this) if you instead use $x_0 = 0, x_1 = -1$, you end up with `karatsuba_mult_poly_alt`.

# Lagrange interpolation theorem and Vandermonde matrices

Recall that the Lagrange interpolation theorem states:

Given rational points $(x_0, y_0), \dotsc, (x_n, y_n)$ such that all $x_i$ are distinct, there exists a unique polynomial $p$ of degree at most $n$ such that $p(x_i) = y_i$ for all $i$.

There are many proofs of this theorem, but for our purposes the most enlightening one is the procedure we used above, i.e. we can build a Vandermonde matrix for $x_0, \dotsc, x_n$, i.e.

$$
A = \left(\begin{matrix}
x_0^0 & x_0^1 & \dotsc & x_0^n \\
x_1^0 & x_1^1 & \dotsc & x_1^n \\
\vdots & \vdots & \ddots & \vdots \\
x_n^0 & x_n^1 & \dotsc & x_n^n
\end{matrix}\right)
$$

which, if it's invertible, has a unique inverse $A^{-1}$. Then apply $A$ to the column matrix $(y_1, \dots, y_n)^T$ to get a column matrix of polynomial coefficients, which then produces a unique polynomial.

Now the key step is inverting $A$, so we must show that $A$ is always invertible. By *waves hands* linear algebra, this is equivalent to showing that $\det A \ne 0$. But fortunately, Vandermonde matrices are well-studied, and a theorem shows that
$$
\det A = (x_0 - x_1) \dotsb (x_{n-1} - x_n) = \prod_{i \ne j} (x_i - x_j),
$$
i.e. the product of all the differences of the $x_i$. (See Wikipedia for a few proofs.) Therefore, $A$ is invertible as long as the $x_i$ are distinct, which they are by assumption.

(Advanced note: note that it does _not_ necessarily hold that Vandermonde matrices are invertible over fields of positive characteristic, even though the expression for the determinant a priori looks like it should be non-zero for distinct $x_i$.)

What about the "almost" Vandermonde matrix that we ended up with, that looks like:

$$
A' = \left(\begin{matrix}
x_0^0 & x_0^1 & \dotsc & 0 \\
x_1^0 & x_1^1 & \dotsc & 0 \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \dotsc & 1
\end{matrix}\right)
$$

Well, by *waves hands* more linear algebra, the determinant of $A'$ reduces to exactly the determinant of the Vandermonde submatrix, which is non-zero.

Note that matrix inversion involves division by the determinant, or something equivalent. Since in this case the determinant is the product of the differences of the $x_i$, this strongly suggests that we want to minimize such differences, i.e. we might want to pick consecutive integers. However, that can only get us so far, as the following proof shows.

# Sketch of proof that division by a multiple of 3 is unavoidable for Toom-Cook

We build a Vandermonde matrix for at least four distinct numbers $x_0, x_1, x_2, x_3$. By the pigeonhole principle, at least two of these numbers have the same residue mod 3. Assume without loss of generality that $x_0 \equiv x_1 \bmod 3$. Then $3 \mid (x_0 - x_1)$, and thus by the expression for the Vandermonde determinant, $3 \mid \det A$.

Even though matrix inversion involves division by the determinant, this alone doesn't prove that division by 3 is unavoidable. For that we must look at the entries of the inverse matrix. By an [explicit formula](https://proofwiki.org/wiki/Inverse_of_Vandermonde_Matrix) for the inverse of the Vandermonde matrix, it has some entries of the form
$$
\dfrac 1 {x_i \, \prod_{\substack {1 \mathop \le m \mathop \le n \\ m \mathop \ne i} } (x_i - x_m) }
$$

Therefore, for $i = 0$, it has an entry of the form
$$
\dfrac 1 {x_0 (x_0 - x_1) \, \prod_{\substack {1 \mathop \le m \mathop \le n \\ m \mathop \ne 0, 1} } (x_0 - x_m) }
$$

Therefore, in the Toom-Cook algorithm this would translate into a division by $x_0 (x_0 - x_1) \dotsb$, which is a multiple of 3.